importing the necessary libaries

In [8]:
import tensorflow as tf
import numpy as np
import cv2
from keras.applications.vgg16 import VGG16
import tkinter as tk
from tkinter import filedialog

model building ,training and testing for sleeping face detection

In [27]:
# Loading the VGGFace model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freezing the base model
base_model.trainable = False

# Adding custom layers to the model
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(2, activation='sigmoid')

model = tf.keras.Sequential([
        base_model,
        global_average_layer,
        prediction_layer
])

# Fine-tuning the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


# Loading the training data
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(r'C:\Users\Shivam Sharma\Documents\Python Scripts\task3\datasets\sleeping_train', target_size=(224, 224), batch_size=32, class_mode='categorical')

# Training the model
history = model.fit(train_generator, epochs=5)

# Evaluating the model on the test dataset
test_generator = train_datagen.flow_from_directory(r'C:\Users\Shivam Sharma\Documents\Python Scripts\task3\datasets\sleeping_test', target_size=(224, 224), batch_size=32, class_mode='categorical')
evaluation = model.evaluate(test_generator)
print('the test accuracy was found to be:',evaluation[0])

Found 300 images belonging to 2 classes.
Epoch 1/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 30s 2s/step - accuracy: 0.6592 - loss: 0.6731
Epoch 2/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 27s 3s/step - accuracy: 0.8806 - loss: 0.6068
Epoch 3/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 25s 3s/step - accuracy: 0.9431 - loss: 0.5553
Epoch 4/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 25s 2s/step - accuracy: 0.9956 - loss: 0.5031
Epoch 5/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 26s 2s/step - accuracy: 0.9982 - loss: 0.4602
Found 80 images belonging to 2 classes.
3/3 ━━━━━━━━━━━━━━━━━━━━ 7s 2s/step - accuracy: 0.7547 - loss: 0.5689
the test accuracy was found to be: 0.5646839141845703


 model building , training and testing for age prediction of the detected face

In [ ]:
# Adding custom layers to the model
age_prediction_layer = tf.keras.layers.Dense(95, activation='sigmoid')

age_model = tf.keras.Sequential([
        base_model,
        global_average_layer,
        age_prediction_layer
])

# Fine-tuning the model
age_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


# Loading the training data
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
age_train_generator = train_datagen.flow_from_directory(r'C:\Users\Shivam Sharma\Documents\Python Scripts\task3\datasets\age_train', target_size=(224, 224), batch_size=32, class_mode='categorical')

# Training the model
age_history = age_model.fit(age_train_generator, epochs=2)

# Evaluating the model on the test dataset
age_test_generator = train_datagen.flow_from_directory(r'C:\Users\Shivam Sharma\Documents\Python Scripts\task3\datasets\age_test', target_size=(224, 224), batch_size=32, class_mode='categorical')
age_evaluation = age_model.evaluate(age_test_generator)
print('the test accuracy was found to be:',age_evaluation[0])

Found 9479 images belonging to 95 classes.
Epoch 1/2
297/297 ━━━━━━━━━━━━━━━━━━━━ 1140s 4s/step - accuracy: 0.0209 - loss: 4.5387
Epoch 2/2
297/297 ━━━━━━━━━━━━━━━━━━━━ 1023s 3s/step - accuracy: 0.0560 - loss: 4.2155
Found 2790 images belonging to 95 classes.
88/88 ━━━━━━━━━━━━━━━━━━━━ 264s 3s/step - accuracy: 0.0551 - loss: 4.1040
the test accuracy was found to be: 4.106809139251709


building the necassary function 

In [99]:
face_cascade = cv2.CascadeClassifier(r'C:\Users\Shivam Sharma\Documents\Python Scripts\task3\haarcascade_frontalface_default (1).xml')
#to extract class name from prediction of the model
def extract_class_name(predictions,train_generator):
      # Geting the class label with the highest predicted probability
      class_label = np.argmax(predictions)

      # Maping the integer label to the corresponding class name
      class_names = train_generator.class_indices
      class_name = [k for k, v in class_names.items() if v == class_label][0]
      return class_name

#image_detection
def is_image(filename):
    #Returns True if the file at the given path is an image, and False otherwise
    try:
        img = cv2.imread(filename)
        if img is not None:
            return True,img
        else:
            return False,None
    except cv2.error:
        return False,None

#vedio_analysis
def vedio_detect_face(vedio_url):
    # Opening the video source (either a webcam or a video file)
    cap = cv2.VideoCapture(vedio_url)
    ved_face=[]
    frames=[]
    # Processing each frame of the video
    while True:
        # Reading the next frame of the video
        ret, frame = cap.read()

        # Checking if the frame was read successfully
        if not ret:
           break
        frames.append(frame)

        # Preprocessing steps
        # 1. Resizing
        frame = cv2.resize(frame, (640, 480))

        # 2. Gray scaling
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # 3. Smoothing
        gray = cv2.GaussianBlur(gray, (7, 7), 0)

        # 4. Histogram equalization
        gray = cv2.equalizeHist(gray)

        # Detection of faces in the grayscale frame
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
        ved_face.append(faces)
    return ved_face,frames



def pred_model(faces,v,img):
    global new_filename,window,class_name,frames,i,num_sp,age
    #cropping the faces from the images
    for f_ace in faces:
      x, y, w, h = f_ace
      face = img[:, y:y+h, x:x+w]
      predictions = model.predict(face)
      #extracting the name of the class
      class_name = extract_class_name(predictions,train_generator)
      if class_name == 'sleeping_person':
        num_sp+=1
        #age prediction of the person
        age_prediction = age_model.predict(face)
        #extracting the name of the class
        age_class_name = extract_class_name(age_prediction,age_train_generator)
        age.append(age_class_name)
    if v== 0:
       if num_sp == 0:
         message = tk.Message(window, text=f"image_url: {new_filename} \n not found any person sleeping in car")
         message.pack()
       else:
          message = tk.Message(window, text=f"image_url: {new_filename} \n found person sleeping in car \n the number of person sleeping are {num_sp} \n the age of the person found sleeping in car are listed below:{age}")
          message.pack()
       return
    else:
       return num_sp,age    

defining the main function

In [131]:
def main():
    global faces,ved_face,new_filename,frames,i,num_sp,age
    is_img,img = is_image(new_filename)
    # initialising the vairable
    age=()#to store the ages of the person sleeping
    num_sp=0
    if (is_img):
        # Preprocessing the image
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces =face_cascade.detectMultiScale(gray)
        v=0
        pred_model(faces,v,img)
    else:
        ved_faces,frames=vedio_detect_face(new_filename)
        v=1
        i=0
        for frame in frames:
           num_sp,age=pred_model(ved_faces[i],v,frame)
           i+=1
        if num_sp==0:
           message = tk.Message(window, text=f"vedio_url: {new_filename} \n not found any person sleeping in car")
           message.pack()
        else:
            message = tk.Message(window, text=f"vedio_url: {new_filename} \n found person sleeping in car \n the number of person sleeping are {num_sp} \n the age of the person found sleeping in car are listed below:{age}")
            message.pack() 

creating window for user interface

In [126]:
window = tk.Tk()
window.title("image classification of a person in vehicle")
window.geometry("500x500")

''

building button for file explorer

In [127]:

label_file_explorer = tk.Label(window, text="")
label_file_explorer.pack()

function for browsing the files

In [128]:
def browseFiles():
    global new_filename,filename
    filename =''
    filename = filedialog.askopenfilename(initialdir="/", title="Select a file",filetypes=(("Image files", "*.png;*.jpg;*.jpeg;*.bmp;*.gif"),
                                                 ("Video files", "*.mp4;*.avi;*.mov;*.mkv;*.flv"), ("All files", "*.*")))
    label_file_explorer.configure(text="File Opened: " + filename)
    new_filename =filename 
    main()

building upload button

In [129]:
label_upload = tk.Label(window, text="Upload a image or vedio file:")
label_upload.pack()

button_upload = tk.Button(window, text="Browse", command=browseFiles)
button_upload.pack()
window.mainloop()